# Find Missing Observations and Imputation

In [1]:
import mysql.connector as M
import pandas as pd
import numpy as np
import json
from datetime import datetime

This work is done by Python3, with packages listed above: 
* `mysql.connector`: the Python3 module name for MySQL-Connector, which is different from that for Python2;
* `pandas`: Python standard package to process data as DataFrames;
* `numpy`: Python package to handle numeric matrices, which is the bridge between Python data structures and Pandas DataFrame;
* `json`: Used for parsing MySQL Database connection configuration from a json format file;
* `datetime`: Package for transforming timestamp to datetime, and vice versa.

MySQL Python3 developer guide: https://dev.mysql.com/doc/connector-python/en/connector-python-versions.html. You can read the "Code Example" section in this link to know how to write SQL statements via Python3 on MySQL database.

## 1. Read in Data from MySQL

First, construction the MySQL database connection. You are free to change the content in "dbconn.json" file in the same directory to your own. As a result, the retrieved data are stored as a Pandas DataFrame named `df_input`.

In [2]:
config = json.load(open('dbconn.json'))["mysql"]
db = M.connect(host = config["host"],
               user = config["user"],
               passwd = config["password"],
               db = config["database"])

query = "SELECT * FROM gdax_history"
df_input = pd.read_sql(query, db)

Here are the stats of this data frame:

In [3]:
print("Number of observations: ", df_input.shape[0])
df_input.head()

Number of observations:  43199


,timestamp,low,high,open,close,volume,utc_datetime,mt_datetime
0,1517788800,8167.90,8199.09,8167.90,8196.48,3.10897,2018-02-05 00:00:00,2018-02-04 17:00:00
1,1517788860,8196.48,8209.46,8196.49,8209.46,10.53460,2018-02-05 00:01:00,2018-02-04 17:01:00
2,1517788920,8209.45,8234.51,8209.45,8234.51,5.29278,2018-02-05 00:02:00,2018-02-04 17:02:00
3,1517788980,8229.75,8259.99,8229.75,8259.99,5.79800,2018-02-05 00:03:00,2018-02-04 17:03:00
4,1517789040,8259.99,8260.00,8260.00,8259.99,2.59747,2018-02-05 00:04:00,2018-02-04 17:04:00


## 2. Construct Another DataFrame of a Complete Set of Minutes

Set the start and end timestamps as the first and last observations(rows) in `df_input`. Also set the interval between two observations as **1 minute**. Then generated a sequence of timestamps from `start_ts` to `end_ts` separated by `ts_step`.

**Notice:** In the numpy function `arange`, parameter `stop` is excluded from the resulting sequence, so I put `end_ts + ts_step` instead of `end_ts` as the stopping timestamp, in order to keep `end_ts + ts_step` inside the generated sequence.

In [4]:
start_ts = df_input.iloc[0]['timestamp']   # Start timestamp
end_ts = df_input.iloc[-1]['timestamp']    # End timestamp
ts_step = 1 * 60    # 1 minute
ts_seq = np.arange(start = start_ts, stop = end_ts + ts_step, step = ts_step)   # stop is (end_ts + ts_step), not end_ts.

Below is the stat of this new DataFrame:

In [5]:
df_tsfull = pd.DataFrame(data = ts_seq, columns = ['timestamp'])
print("Number of observations if no missing data: ", df_tsfull.shape[0])
df_tsfull.head()

Number of observations if no missing data:  43200


,timestamp
0,1517788800
1,1517788860
2,1517788920
3,1517788980
4,1517789040


By comparing the number of observations of the two data frames, we can see that there is one observation with no data.

We can further generate the corresponding UTC date and time column for each timestamp, by defining the following function `timestamp_to_utcstr` (I simply copied its source code from `gdax_history.py` file):

In [6]:
def timestamp_to_utcstr(ts):
    utc_str = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S")
    return utc_str

df_tsfull['utc_datetime'] = df_tsfull.apply(lambda row: timestamp_to_utcstr(row['timestamp']), axis = 1)
df_tsfull.head()

,timestamp,utc_datetime
0,1517788800,2018-02-05 00:00:00
1,1517788860,2018-02-05 00:01:00
2,1517788920,2018-02-05 00:02:00
3,1517788980,2018-02-05 00:03:00
4,1517789040,2018-02-05 00:04:00


## 3. Join the Two Data Frames

We now have two data frames: `df_input` from the original table, and `df_tsfull` of a complete set of timestamps.

To achieve a table of a complete set of timestamps and values of all the other columns, we need a join on the two data frames:
1. If putting `df_input` as the left data frame, and `df_tsfull` as the right one, this is a RIGHT JOIN, because we want to maintain all the rows in `df_tsfull` as it contains a complete set of timestamps.
2. We need to set the index of both tables as `timestamp`, in order to perform this join on `timestamp`. You can tell the index column of a Pandas DataFrame by checking the column of a bold font in its summary or head result.
3. In the `join` function below, parameter `lsuffix` and `rsuffix` are used when we have columns of the same name from two tables. In this case, it is `utc_datetime`. So in the resulting DataFrame, the one from left DataFrame is renamed as `utc_datetime1`, while the other is renamed to `utc_datetime2`.

In [7]:
df_join = df_input.set_index('timestamp').join(df_tsfull.set_index('timestamp'), how = 'right', 
                                               lsuffix = '1', rsuffix = '2')
df_join.head()

,low,high,open,close,volume,utc_datetime1,mt_datetime,utc_datetime2
timestamp,,,,,,,,
1517788800,8167.90,8199.09,8167.90,8196.48,3.10897,2018-02-05 00:00:00,2018-02-04 17:00:00,2018-02-05 00:00:00
1517788860,8196.48,8209.46,8196.49,8209.46,10.53460,2018-02-05 00:01:00,2018-02-04 17:01:00,2018-02-05 00:01:00
1517788920,8209.45,8234.51,8209.45,8234.51,5.29278,2018-02-05 00:02:00,2018-02-04 17:02:00,2018-02-05 00:02:00
1517788980,8229.75,8259.99,8229.75,8259.99,5.79800,2018-02-05 00:03:00,2018-02-04 17:03:00,2018-02-05 00:03:00
1517789040,8259.99,8260.00,8260.00,8259.99,2.59747,2018-02-05 00:04:00,2018-02-04 17:04:00,2018-02-05 00:04:00


The generated DataFrame is shown above. And you can see that its index column is `timestamp`, which is in bold font.

We can make the DataFrame simpler:
* Drop columns `mt_datetime` and `utc_datetime1`. Notice that we do need to maintain `utc_datetime2` as it contains the UTC date time information on the complete set of timestamps, while `utc_datetime1` is not.
* Rename `utc_datetime2` to `utc_datetime`.
* Reset the index, so that `timestamp` is not the index, but a normal column. This step is optional, if you like treating `timestamp` as the index.

In [8]:
df_join = df_join.drop(columns = ['mt_datetime', 'utc_datetime1'])
df_join = df_join.rename(columns = {'utc_datetime2': 'utc_datetime'})
df_join = df_join.reset_index()
df_join.head()

,timestamp,low,high,open,close,volume,utc_datetime
0,1517788800,8167.90,8199.09,8167.90,8196.48,3.10897,2018-02-05 00:00:00
1,1517788860,8196.48,8209.46,8196.49,8209.46,10.53460,2018-02-05 00:01:00
2,1517788920,8209.45,8234.51,8209.45,8234.51,5.29278,2018-02-05 00:02:00
3,1517788980,8229.75,8259.99,8229.75,8259.99,5.79800,2018-02-05 00:03:00
4,1517789040,8259.99,8260.00,8260.00,8259.99,2.59747,2018-02-05 00:04:00


Now we have a clean structure of this DataFrame, and we can find the observation with missing data as follows:

In [9]:
df_join.loc[df_join['volume'].isnull()]

,timestamp,low,high,open,close,volume,utc_datetime
33633,1519806780,NaN,NaN,NaN,NaN,NaN,2018-02-28 08:33:00


## Imputation

We can perform imputation on this observation. Pandas has a function `interpolate` to do imputation on numeric variables using interpolation method. It also provides other methods on imputation. For details, please check: https://pandas.pydata.org/pandas-docs/stable/missing_data.html.

In [10]:
df_imp = df_join
df_imp[['low', 'high', 'open', 'close', 'volume']] = df_imp[['low', 'high', 'open', 'close', 'volume']].interpolate()
df_imp.iloc[33633]

timestamp                1519806780
low                         10740.8
high                        10755.9
open                        10755.9
close                       10740.8
volume                      4.02093
utc_datetime    2018-02-28 08:33:00
Name: 33633, dtype: object

## Write to a New Table

Now we are ready to write this imputed complete DataFrame back to database.

We may think of creating a new table for this data:

In [11]:
table_desc = (
    "CREATE TABLE `gdax_history_imputed` ("
    "  `timestamp` int(10) UNSIGNED NOT NULL,"
    "  `low` float NOT NULL,"
    "  `high` float NOT NULL,"
    "  `open` float NOT NULL,"
    "  `close` float NOT NULL,"
    "  `volume` float NOT NULL,"
    "  `utc_datetime` datetime NOT NULL,"
    "  PRIMARY KEY (`timestamp`)"
    ") ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin")

cur = db.cursor()
try:
    cur.execute(table_desc)
except M.Error as e:
    print(e.msg)
else:
    print("OK.")
    
cur.close()

OK.


True

The new table name is `gdax_history_imputed`. The SQL statement in `table_desc` above is simplied copied from `cryptocurrency.sql` in this directory and modified.

Pandas has a function `to_sql` for DataFrame, but it uses a different Python package for connection to database. To be consistent in this notebook, I defined my own write function via MySQL-Connector package, which is quite similar to the one in `gdax_history.py`:

In [12]:
def write_to_db(df, db):
    if len(df) == 0:
        return

    cur = db.cursor()
    df_hash = df.reset_index(drop = True).to_dict()
    try:
        for i in range(df.shape[0]):
            cur.execute(
                """INSERT INTO gdax_history_imputed (timestamp, low, high, open, close, volume, utc_datetime)
                VALUES (%s, %s, %s, %s, %s, %s, %s)""",
                (df_hash['timestamp'][i], df_hash['low'][i], df_hash['high'][i], df_hash['open'][i],
                 df_hash['close'][i], df_hash['volume'][i], df_hash['utc_datetime'][i]))
        db.commit()
    except M.Error as e:
        print(e.msg)
        db.rollback()
    else:
        print("Successfully write ", df.shape[0], " entries to table.")
        
    cur.close()

The input of `write_to_db` is `df`, a Pandas DataFrame, and `db`, MySQL connection. In order to iterate each row in `df`, I first transform it into a Python dictionary (`df_hash`), with column names being the keys, and rows being the values.

The reason that Pandas uses columns instead of rows as the keys, is that this is the standard way that NoSQL database handling big data to store the data. On the contrary, traditional relational database, such as MySQL, uses rows as the keys, but columns as the values.

Finally, use this `write_to_db` function to write `df_imp` DataFrame to database. 

In [13]:
num_obs = df_imp.shape[0]
num_per_turn = 10000
start_idx = 0
while start_idx < num_obs:
    if num_obs - start_idx < num_per_turn:
        df_slice = df_imp.iloc[start_idx:(num_obs - 1)]
        write_to_db(df_slice, db)
        start_idx = num_obs
    else:
        df_slice = df_imp.iloc[start_idx:(start_idx + num_per_turn)]
        write_to_db(df_slice, db)
        start_idx = start_idx + num_per_turn


Successfully write  10000  entries to table.
Successfully write  10000  entries to table.
Successfully write  10000  entries to table.
Successfully write  10000  entries to table.
Successfully write  3199  entries to table.
